In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-capstone/customer_capstone_project.csv


# 3. Sentiment Classification

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
## Read our final cleaned dataset 
df = pd.read_csv("/kaggle/input/customer-capstone/customer_capstone_project.csv")
df.head(3)

,name,city,Business_stars,review_count,is_open,categories,attributes,review_stars,review_useful,review_funny,review_cool,review_text,review_cleaned_text,review_preprocessed_text,tip_compliment_count,tip_text,tip_cleaned_text,tip_preprocessed_text,year,month,review_sentiment,date,season
0,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,0.0,good ambience\n\ngood service\n\nOrdered: \n\n...,good ambience\n\ngood service\n\nordered \n\nf...,goodambience\n\ngoodservice\n\norder\n\nfishpe...,0.0,"Started with a few appetizers - all good, thou...",started with a few appetizers all good though...,startappetizer goodindianpancakestandmaincours...,2012.0,11.0,0.415278,2012-11-01,Rainy
1,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,0.0,good ambience\n\ngood service\n\nOrdered: \n\n...,good ambience\n\ngood service\n\nordered \n\nf...,goodambience\n\ngoodservice\n\norder\n\nfishpe...,0.0,Awesome!!!,awesome,awesome,2013.0,12.0,0.415278,2013-12-01,Winter
2,Spice 28,Philadelphia,4.0,822.0,1.0,"['Asian Fusion', 'Restaurants', 'American (New...","{'RestaurantsGoodForGroups': 'True', 'WiFi': ""...",4.0,1.0,0.0,0.0,good ambience\n\ngood service\n\nOrdered: \n\n...,good ambience\n\ngood service\n\nordered \n\nf...,goodambience\n\ngoodservice\n\norder\n\nfishpe...,0.0,Good food. There were things I liked and disli...,good food there were things i liked and disliked,goodfoodthinglikedislike,2013.0,3.0,0.415278,2013-03-01,Spring


In [4]:
# df.review_cleaned_text[3]
# df.review_sentiment[3]
df.duplicated().sum()

0

In [5]:
## Convert Sentiment Scores to Categories

# Function to convert sentiment score to categories
def convert_sentiment(score):
    if score <= -0.1:
        return 'negative'
    elif -0.1 < score <= 0.1:
        return 'neutral'
    else:
        return 'positive'

In [6]:
# Apply the function to the sentiment_score column
df['sentiment_label'] = df['review_sentiment'].apply(convert_sentiment)

# Check the result
df_sentiment = df[['review_cleaned_text', 'sentiment_label']]
df_sentiment.tail()

,review_cleaned_text,sentiment_label
10335,i ate here a few times before and was very ple...,positive
10336,best food ever have been trying hard to find ...,positive
10337,best food ever have been trying hard to find ...,positive
10338,awesome dive bar citywides here looooove love...,positive
10339,i had the crispy chicken wrap with dijon and f...,positive


In [7]:
df_sentiment

,review_cleaned_text,sentiment_label
0,good ambience\n\ngood service\n\nordered \n\nf...,positive
1,good ambience\n\ngood service\n\nordered \n\nf...,positive
2,good ambience\n\ngood service\n\nordered \n\nf...,positive
3,good ambience\n\ngood service\n\nordered \n\nf...,positive
4,good ambience\n\ngood service\n\nordered \n\nf...,positive
...,...,...
10335,i ate here a few times before and was very ple...,positive
10336,best food ever have been trying hard to find ...,positive
10337,best food ever have been trying hard to find ...,positive
10338,awesome dive bar citywides here looooove love...,positive


**Model Selection**

For sentiment classification, we will be using BERT from the Hugging Face transformers library.

In [8]:
# Import all the neccessary libraries

from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

In [9]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
train_df, test_df = train_test_split(df_sentiment, test_size=0.2, random_state=42)

In [11]:
# Tokenize the text
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128)

In [12]:
# Apply tokenization
train_encodings = tokenizer(list(train_df['review_cleaned_text']), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_df['review_cleaned_text']), padding=True, truncation=True, max_length=128)

**Convert Data into PyTorch Datasets**

You need to convert the tokenized data into PyTorch datasets for training the model.

In [13]:
import torch
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convert data to PyTorch datasets
train_labels = [1 if label == 'positive' else 0 for label in train_df['sentiment_label']]  # Binary labels
test_labels = [1 if label == 'positive' else 0 for label in test_df['sentiment_label']]

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

# BERT Model

In [15]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # Output directory
    num_train_epochs=3,               # Number of training epochs
    per_device_train_batch_size=8,    # Batch size per device during training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    warmup_steps=500,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # Strength of weight decay
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",      # Evaluation strategy to use
    save_strategy="epoch",            # Save strategy must match the evaluation strategy
    load_best_model_at_end=True,      # Load the best model when finished training
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-tune the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.114700,0.120844
2,0.025500,0.079994
3,0.000300,0.103557


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1551, training_loss=0.12770566255221463, metrics={'train_runtime': 553.5093, 'train_samples_per_second': 44.834, 'train_steps_per_second': 2.802, 'total_flos': 1632340987453440.0, 'train_loss': 0.12770566255221463, 'epoch': 3.0})

# Evaluate Model (BERT)

In [16]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Evaluate the model
predictions, labels, _ = trainer.predict(test_dataset)

# Convert logits to predicted labels
predicted_labels = predictions.argmax(axis=-1)

# Calculate metrics
accuracy = accuracy_score(labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(labels, predicted_labels, average='binary')

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Accuracy: 0.9807
Precision: 0.9853
Recall: 0.9911
F1-Score: 0.9882


# RoBERTa

In [17]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [18]:
# Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the text for RoBERTa input
train_encodings = tokenizer(list(train_df['review_cleaned_text']), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_df['review_cleaned_text']), padding=True, truncation=True, max_length=128)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
# Convert labels to binary (positive=1, negative=0)
train_labels = [1 if label == 'positive' else 0 for label in train_df['sentiment_label']]  # Binary labels
test_labels = [1 if label == 'positive' else 0 for label in test_df['sentiment_label']]

In [21]:
# Convert data to PyTorch datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [22]:
# Load pre-trained RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # Output directory
    num_train_epochs=3,               # Number of training epochs
    per_device_train_batch_size=8,    # Batch size per device during training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    warmup_steps=500,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # Strength of weight decay
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",      # Evaluate every epoch
    save_strategy="epoch",            # Save model every epoch
    load_best_model_at_end=True,      # Load the best model when finished training
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-tune the model
trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather alo

Epoch,Training Loss,Validation Loss
1,0.249300,0.242949
2,0.104900,0.131745
3,0.043400,0.091849


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1551, training_loss=0.1910823626892742, metrics={'train_runtime': 567.9482, 'train_samples_per_second': 43.694, 'train_steps_per_second': 2.731, 'total_flos': 1632340987453440.0, 'train_loss': 0.1910823626892742, 'epoch': 3.0})

# Evaluate Model (RoBERTa)

In [23]:
# Evaluate the model
predictions, labels, _ = trainer.predict(test_dataset)

# Convert logits to predicted labels
predicted_labels = predictions.argmax(axis=-1)

# Calculate metrics
accuracy = accuracy_score(labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(labels, predicted_labels, average='binary')

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Accuracy: 0.9821
Precision: 0.9842
Recall: 0.9941
F1-Score: 0.9891


# T5 Model

In [26]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

In [33]:
# Define a function to preprocess the data
def preprocess_data(df):
    return [
        {
            'input_text': f"Classify sentiment: {text}",
            'target_text': 'positive' if sentiment == 'positive' else 'negative'
        }
        for text, sentiment in zip(df['review_cleaned_text'], df['sentiment_label'])
    ]

# Preprocess the data
train_data = preprocess_data(train_df)
test_data = preprocess_data(test_df)

# Convert to datasets
train_dataset = Dataset.from_pandas(pd.DataFrame(train_data))
test_dataset = Dataset.from_pandas(pd.DataFrame(test_data))

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Tokenize the data
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input_text'], padding=True, truncation=True, max_length=128)
    labels = tokenizer(examples['target_text'], padding=True, truncation=True, max_length=32)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Load the pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small',num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()


Map:   0%|          | 0/8272 [00:00<?, ? examples/s]

Map:   0%|          | 0/2068 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.346800,0.116349
2,0.094900,0.087562
3,0.079800,0.081088


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1551, training_loss=0.17046615756149833, metrics={'train_runtime': 266.3863, 'train_samples_per_second': 93.158, 'train_steps_per_second': 5.822, 'total_flos': 839660536332288.0, 'train_loss': 0.17046615756149833, 'epoch': 3.0})

In [43]:
from sklearn.metrics import accuracy_score

output = trainer.predict(test_dataset)

# Access the predictions (logits) from the output
logits = output.predictions

# Check the structure of logits if it is a tuple
# If logits are a tuple, extract the actual logits tensor
if isinstance(logits, tuple):
    logits = logits[0]

# Apply argmax to the logits to get the predicted class labels
predicted_labels = logits.argmax(axis=-1)  # Use 'axis=-1' to get the predicted class


# Flatten the labels and predicted labels if they are multi-output/multilabel
predicted_labels_flat = predicted_labels.flatten()
true_labels_flat = output.label_ids.flatten()

# Now calculate the metrics
accuracy = accuracy_score(true_labels_flat, predicted_labels_flat)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels_flat, predicted_labels_flat)

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision.mean():.4f}")
print(f"Recall: {recall.mean():.4f}")
print(f"F1-Score: {f1.mean():.4f}")



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Accuracy: 0.9734
Precision: 0.9538
Recall: 0.9228
F1-Score: 0.9368


# 4. Topic Modeling